In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import io
# from google.colab import files
import os
import datetime
import time
import sys
import math
#from lxml import etree
from io import StringIO, BytesIO
import seaborn as sns
import yfinance as yf

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ten_q_with_ticker_1 = pd.read_csv('ten_q_with_ticker/ten_q_with_ticker_1.csv')

In [ ]:
def obtain_mda_and_prices_data(df):
    current_set = df
    for index, row in current_set.iterrows():
        try:
              
            starttime = datetime.datetime.now()
            # print("Processing row number", index)
            
            ###### PARSING 10Q SECTION TO EXTRACT MDA ######
            ten_q_raw = requests.get(row["sec_url"]).text

            ## parse the raw text and get rid of tags
            soup = BeautifulSoup(ten_q_raw, 'lxml')
            notags = soup.getText()

            ## use regex to extract MDA
            #### first splitting on all mentions of "Item 2. Management's Discussion and Analysis" ignoring case sensitivity
            #### and taking the last element of split list
            start_of_mda = re.split("Item[\s]*2[\s]*[.\-–:]+[\s]*[.\-–]*[\s]*Management[\S]?s?[\s]*Discussion[\s]*and[\s]*Analysis[\s]*", 
                          notags, 
                          flags = re.IGNORECASE)[-1]
                    
            #### then taking that last element from above and splitting on "Item 3. Quantitative and Qualitative Disclosures About Market Risk" ignoring case sensitivity
            #### and taking the first element of this split list to get the full MDA section
            full_mda = re.split("Item[\s]*[34][\s]*[.\-–:]*[\s]*[.\-–]*[\s]*(Quantitative[\s]*and[\s]*Qualitative[\s]*disclosures[\s]*|Controls[\s]*and[\s]*Procedures[\s]*)", 
                                start_of_mda, 
                                flags = re.IGNORECASE)[0]

            mda_length = len(full_mda)
            full_ten_q_length = len(notags)
            mda_percentage = mda_length/full_ten_q_length

            current_set.loc[index, 'mda_raw_text'] = full_mda
            current_set.loc[index, 'ending_of_mda'] = full_mda[-100:-1]
            current_set.loc[index, 'mda_length'] = mda_length
            current_set.loc[index, 'full_ten_q_length'] = full_ten_q_length
            current_set.loc[index, 'mda_percentage'] = mda_percentage
            endtime = datetime.datetime.now()

            # print("Current row took", endtime - starttime)

            ###### (COMPLETED) PARSING 10Q SECTION TO EXTRACT MDA ######



            ####### OBTAINING STOCK PRICE DATA #######
            #### closing price on filing date and opening price on next trading day
            row_ticker = row["ticker"]
            row_date = datetime.datetime.strptime(row["accepted_filing_date"], "%Y-%m-%d").date() 
            prices_for_timewindow = yf.Ticker(row_ticker).history(start = row_date, end = row_date + datetime.timedelta(days = 6)).reset_index() ## setting time window to 6 days because this will ensure that we get two consecutive trading days (avoiding weekends and holidays)
            prices_for_timewindow = prices_for_timewindow.iloc[:2,:]

            if prices_for_timewindow.shape[0] > 1:          ## only fill in values in columns if pricing data is available for these dates
                current_set.loc[index, "trading_date_closest_to_filing_date"] = prices_for_timewindow.loc[0,"Date"] 
                current_set.loc[index, "closing_price_on_filing_date"] = prices_for_timewindow.loc[0,"Close"]

                current_set.loc[index, "next_trading_date"] = prices_for_timewindow.loc[1,"Date"]
                current_set.loc[index, "opening_price_on_next_trading_date"] = prices_for_timewindow.loc[1,"Open"]


            #### opening price on x days later
            days_after = [7,30]
            for window in days_after:
                prices_for_timewindow = yf.Ticker(row_ticker).history(start = row_date, end = row_date + datetime.timedelta(days = window + 1)).reset_index()
                prices_for_timewindow_rows = prices_for_timewindow.shape[0]

                if prices_for_timewindow_rows > 1:         ## also only fill in values in columns if pricing data is available
                    current_set.loc[index, "trading_date_" + str(window) + "_days_after"] = prices_for_timewindow.loc[prices_for_timewindow_rows - 1,"Date"]
                    current_set.loc[index, "opening_price_" + str(window) + "_days_after"] = prices_for_timewindow.loc[prices_for_timewindow_rows - 1,"Open"]

            ####### (COMPLETED) OBTAINING STOCK PRICE DATA #######


            # no need to sleep because each iteration takes about 1.5 seconds (and the limit is 10 calls per second)
            time.sleep(0.1)
            if index%100 == 0:
                print("Just finished row number", index)
                print(datetime.datetime.now())
                print()
        except:
            print("Hit an error while processing row", str(index))
            print("Continuing to next row")
            continue

    return current_set

In [ ]:
def chunk_process_data_and_write(df, subset, chunks):
    rows_in_df = df.shape[0]
    chunk_size = math.floor(rows_in_df / chunks)

    for i in range(chunks):
        if i != chunks - 1:
            starting = i * chunk_size
            ending = chunk_size * (i+1)
        else:
            starting = i * chun_size
            ending = -1

        next_dataset_to_process = df[starting:ending]
        output = obtain_mda_and_prices_data(next_dataset_to_process)
        
        print("Writing to CSV: " + 'data_augmented_' + str(subset) + '_' + str(starting) + 'to' + str(ending) + '.csv')
        output.to_csv('data_augmented_' + str(subset) + '_' + str(starting) + 'to' + str(ending) + '.csv', index = False)

In [ ]:
chunk_process_data_and_write(ten_q_with_ticker_1, 1, 10)